# Molecular Dynamics Asignment: Barrier Crossing In 1D
Authors: *Stefan Hervø-Hansen*<sup>🐺</sup>, *Vidar Aspelin*<sup>🦡</sup>, and *Samuel Stenberg*<sup>🦄</sup>.  
Division of Theoretical Chemistry, Lund University. 

Electronic Address:  
🐺 stefan.hervo_hansen@teokem.lu.se  
🦡 vidar.aspelin@teokem.lu.se  
🦄 samuel.stenberg@teokem.lu.se

<figure>
    <img src="include/black_swan.png" width=280 />
</figure>
<center>How does the title and asignment relate to the picture? The answer will be more aparent by the end of the asignment.</center>

## Intended learning outcomes
1. The effects on the time step on the total energy fluctuations
2. Transformation of energy (potential vs kinetic energy)
3. Concept of phase space and available microstates in different ensembles
4. Sampling differences between MD and MC

## Theory and Questions for Reflection
Consider the movement of a single particle that moves on a 1D potential energy surface with the following functional form:
$$
U(x)=
\begin{cases}
\epsilon B x^2, & \text{if}\ x \leq 0.0\\
\epsilon x_{\circ}^2 \left[ 1 - \cos \left( \frac{2\pi x}{x_\circ} \right)\right], & \text{if}\ 0.0 \leq x \leq 1.0\\
\epsilon B (x_{\circ}-x)^2, & \text{if}\ x \geq 1.0
\end{cases}
\tag{eq. 1}
$$
The energy, force and the derivative of the force should be continuous functions of the position $x$ with $\epsilon$ > 0 and $x_\circ$ is a standard position set to 1 for units to cancel, here $x_\circ=1 \ \mathrm{nm}$. We want to investigate this system using Molecular dynamics.

**Question 1: Briefly explain Molecular dynamics (include the concepts Newton's equations of motion, trajectory, integrator, time step, NVE ensemble, energy)**  

In order to propagate positions and velocities in time, we need information about the acceleration, which is obtained from the forces. To make sure that our system is stable and can be propagated correctly, we need to make the potential, the force, and the derivative of the force (with respect to x) continuous.

<figure>
<img src="include/MC_vs_MD_potential.png" width=800 />
</figure>
<center>A comparison between a non-continuous potential (hard spheres) and a continuous potential (Weeks-Chandler-Andersen.)</center>

__Question 2: Derive an expression for *B* so that the potential, force and derivative of the force becomes continuous and plot/sketch the energy landscape. Also insert your derived expression for *B* in the code containing the function Potential below.__  

The program pymd is provided that integrates the equation of motion of the particle on the potential energy surface, starting at the position 0 ($x(t=0)=0$), using either molecular dynamics and methods thereof or Monte Carlo simulation. The program functionalities has been listed below:  
<ul style="list-style-type:none;">
  <li>Molecular dynamics, NVE ensemble</li>
  <li>Molecular dynamics with Andersen thermostat</li>
  <li>Molecular dynamics with Berendsen thermostat</li>
  <li>Monte Carlo </li>
</ul>  
For the functionalities with thermostats, we are approximating the NVT ensemble using two different thermostats, in particular the Andersen thermostat and Berendsen thermostat which can be categorized as a stochastic method and velocity rescale method respectively.


Before conducting simulations, review the part of the code (pymd.py) containing the Velocity Verlet integrator by executing the cell below, and try to understand what is happening.

In [ ]:
!sed -n 34,39p pymd.py

In [ ]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import random
import pymd as sim # This is where we import the program used in this laboratory session

In [ ]:
# Function defining the potential (eq.1) and force (eq.2) 
eps = 15
B = # FINISH THE CODE: Add expression for B
def Potential(x):
    '''Function returning energy and force depending on the position, x, on the 1D energy surface.'''
    if x <= 0.0:
        U = eps *  B * x**2
        F = eps * -2 * B * x
    elif x >= 1.0:
        U = eps * B * (x-1.0)**2
        F = eps * -2 * B * (x-1.0)
    else:
        U = eps * (1.0 - np.cos(2*np.pi*x))
        F = eps * -2.0 * np.pi * np.sin(2.0*np.pi*x)
    return U, F      

In [ ]:
# Plot the energy and force here.
# Plot energy:
x = np.arange(-1, 2, 0.01)
y = [Potential(x_)[0] for x_ in x]
plt.plot(x, y)
plt.xlabel('$x$ (nm)')
plt.ylabel('$U$(x) (kJ/mol)')

# FINISH THE CODE: Plot the force. Check so that the force is everywhere differentiable.

**Question 3: Conduct short simulations of the NVE ensemble using different time steps but the same simulation time and comment on the differences in energy profiles (potential, kinetic and total energy). Elaborate on the differences you observe!**

Newton's equations tell us that if we have a set of particle positions and particle velocities at an arbitrary time, it is possible to predict the positions and velocities at any one particular instant in time. Therefore, classical dynamics of a $N$-particle system can be expressed by specifying the full set of 6$N$ variables ($2N$ for each dimension). The $6N$ variables can be regarded as a single point in a $6N$-dimensional space named the _phase space_. The phase space is a Cartesian space and solving Newton's equations yields a trajectory in phase space. Consequently, classical motion can be described by the motion of a point along a trajectory in phase space, with each point in phase space representing a possible microstate. 

To be able to visualize phase space, we will in the following plot the phase space for 1 particle in 1 dimension.

In [ ]:
# SIMULATION SETTINGS, MD-simulations (NVE)
Temperature = 300                                 # [K]
Nparticles = 100                                  # Number of independent particles
Tstep = 0.001                                      # [ps]
SimTime = 1                                       # Simulation time [ps]
Nstep = int(SimTime/Tstep)

In [ ]:
nve = sim.Simulator(Nstep, Nparticles, Tstep, Temperature)

for j in range(nve.Nparticles): # Particle loop  
    nve.Velocity[j] = 1 # md.RandomVelocity(Temperature, m)


for i in range(Nstep): # Main Loop
    for j in range(nve.Nparticles): # Particle loop  
        nve.VelocityVerlet_NVE(j, Potential) # integrate motion of particle j
    nve.SampleData() # sample data for step i

### Data visualization

In [ ]:
# Plotting potential, kinetic and total energy as a function of simulation time
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(nve.Data_time[:], np.sum(nve.Data_Epot[:], axis=1), label=r'Potential energy', lw=3, alpha=0.7)
ax.plot(nve.Data_time[:], np.sum(nve.Data_Ekin[:], axis=1), label=r'Kinetic energy', lw=3, alpha=0.7)
ax.plot(nve.Data_time[:], np.sum(nve.Data_Etot[:], axis=1), label=r'Total energy', lw=3, alpha=0.7)

# Graphics settings
ax.minorticks_on()
ax.tick_params(axis='both',which='minor',length=5,width=2,labelsize=18)
ax.tick_params(axis='both',which='major',length=8,width=2,labelsize=18)
ax.legend(fontsize=16, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

ax.set_title(r'Molecular Dynamics Stability', fontsize=22)
ax.set_xlabel(r'Time (ps)', fontsize=22, labelpad=20)
ax.set_ylabel(r'Energy (kJ/mol)', fontsize=22, labelpad=20)

plt.tight_layout()
plt.show()

**Question 4: What do you expect the phase space trajectories for the 1 particle to look like in terms of density and shape for the following situations, given a (i) Harmonic potential and (ii) the potential given in Eq. 1? _Hint_: The first term in Eq. 1 is a harmonic oscillator. How would the shape of phase space change if we added more energy to the NVE ensemble?**
1. *NVE* ensemble.
2. *NVT* ensemble.

**Question 5: Using the simulations methods (NVE, NVT and MC) for low temperatures (150 < T < 300 K), comment on the following:**

*Note*: When simulating in MC, we no longer have velocities. This means that one dimension in phase space, the velocity dimension, dissapears and we are left with only the positions (the *configurational* dimension). So when talking about phase space in MC, we say configurational space rather than phase space. For the *NVT* ensemble, we will only be using the Andersen thermostat in this lab exercise.

1. Does the result correspond with what you expected in terms of the density and shape? Comment on any differences. 
2. Why does the phase space distribution for the *NVE* and *NVT* ensemble look so different?
3. Why does the configurational space distribution of the MC scheme look so much different compared to the phase space distribution of the MD schemes at reasonably low temperatures (<10000K)?
4. See cell with question 5.4

In [ ]:
# A look into pymd for the Andersen thermostat random velocity asignment
!sed -n 84,89p pymd.py
!echo " "
!sed -n 110,112p pymd.py

In [ ]:
# SIMULATION SETTINGS, MD-simulations (NVE and NVT)
Temperature = 300                                 # [K]
Nparticles = 10
# Number of independent particles
Tstep = 0.001                                      # [ps]
SimTime = 500                                       # Simulation time [ps]
Nstep = int(SimTime/Tstep)                        # Calculated number of simulation steps 
MU = 1                                            # Collision frequency (used for NVT-ensemble simulation)

In [ ]:
# MD-simulations with NVE and NVT

nve = sim.Simulator(Nstep, Nparticles, Tstep, Temperature)
nvt = sim.Simulator(Nstep, Nparticles, Tstep, Temperature, MU)

for j in range(nve.Nparticles): # Particle loop  
    nve.Velocity[j] = nve.RandomVelocity(Temperature, 1) #Set random velocity from MB distribution
    nvt.Velocity[j] = nvt.RandomVelocity(Temperature, 1)

for i in range(Nstep): # Main Loop for NVE simulation
    for j in range(nve.Nparticles): # Particle loop  
        nve.VelocityVerlet_NVE(j, Potential) # integrate motion of particle j
    nve.SampleData() # sample data for step i
    
for i in range(Nstep): # Main Loop for NVT simulation
    for j in range(nvt.Nparticles): # Particle loop  
        nvt.VelocityVerlet_NVE(j, Potential) # integrate motion of particle j
    nvt.Thermostat_And() # Applying thermostat to simulate in NVT
    nvt.SampleData() # sample data for step i

### Data visualization

In [ ]:
# Plotting phase space distributions for MD simulations (NVE and NVT)
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize=(12,6))

ax1.hist2d(nve.Data_pos.ravel(), nve.Data_vel.ravel(), normed=True, bins=100)    
    
# Graphics settings
ax1.minorticks_on()
ax1.tick_params(axis='both',which='minor',length=5,width=2,labelsize=18)
ax1.tick_params(axis='both',which='major',length=8,width=2,labelsize=18)

ax1.set_title(r'Phase Space Distribution, NVE', fontsize=22)
ax1.set_ylabel(r'Velocity (nm/ps)', fontsize=22, labelpad=20)
ax1.set_xlabel(r'Position (nm)', fontsize=22, labelpad=20)

ax2.hist2d(nvt.Data_pos.ravel(), nvt.Data_vel.ravel(), normed=True, bins=100)    
    
# Graphics settings
ax2.minorticks_on()
ax2.tick_params(axis='both',which='minor',length=5,width=2,labelsize=18)
ax2.tick_params(axis='both',which='major',length=8,width=2,labelsize=18)

ax2.set_title(r'Phase Space Distribution, NVT', fontsize=22)
ax2.set_ylabel(r'Velocity (nm/ps)', fontsize=22, labelpad=20)
ax2.set_xlabel(r'Position (nm)', fontsize=22, labelpad=20)

fig.tight_layout()

In [ ]:
# SIMULATION SETTINGS, MC-simulation
Temperature = 300                                 # [K]
Nparticles = 10                                  # Number of independent particles
Tstep = 0.001                                     # [ps]
SimTime = 500                                       # Simulation time [ps]
Nstep = int(SimTime/Tstep)                        # Calculated number of simulation steps 
dp = 1                                            # Displacement parameter for MC

In [ ]:
# MC-simulations

mc = sim.Simulator(Nstep, Nparticles, Tstep, Temperature, dp=dp)

for i in range(Nstep): # Main Loop for NVE simulation
    for j in range(mc.Nparticles): # Particle loop  
        mc.MonteCarlo(j, Potential) # integrate motion of particle j
    mc.SampleData() # sample data for step i

In [ ]:
# Plotting phase configurational space distribution for MC-simulation
fig, ax = plt.subplots(figsize=(7,6))

ax.hist(mc.Data_pos.ravel(), density=True, bins=100)


    
# Graphics settings
ax.minorticks_on()
ax.tick_params(axis='both',which='minor',length=5,width=2,labelsize=18)
ax.tick_params(axis='both',which='major',length=8,width=2,labelsize=18)

ax.set_title(r'Configurational Space Distribution',fontsize=22)
ax.set_ylabel(r'Probability', fontsize=22, labelpad=20)
ax.set_xlabel(r'Position (nm)', fontsize=22, labelpad=20)

**5.4.** Show that the MD data obtained using the Andersen thermostat matches the analytical probability distribution for velocities and positions for the harmonic oscillator. Use the code provided below, and insert the missing expression for the partition function. *Hint*:
$$
\int_{-\infty}^{\infty} e^{-a(x+b)^2} \mathrm{d}x = \sqrt{\frac{\pi}{a}}
$$

In [ ]:
################################################
#                QUESTION 5.4                  #
# Boltzman distribution: Potential energy part #
################################################

plt.hist(nvt.Data_pos.ravel(), density=True, bins=1000)

# FINISH THE CODE: START
# Potential energy partition function:
partition_function_PE =  #insert partition function here

def probability_PE(m, Temperature, x):  # Probability as a function of position 
    return #insert probability here

# FINISH THE CODE: END

x = np.arange(-0.2, 0.2, 0.0001)
plt.plot(x, probability_PE(1, Temperature, x), 'r--', linewidth=3)
plt.xlabel(r'$\bf{x}\ \mathrm{(nm)}$')
plt.ylabel(r'$\bf{P(x)}$')
plt.show()
# Self-Check: The sum of all probabilities should always equal 1!
print('The area under the curve is equal to: {:03.2f}'.format(np.trapz(probability_PE(1,Temperature,x), x)))

In [ ]:
##############################################
#                QUESTION 5.4                #
# Boltzman distribution: Kinetic energy part #
##############################################
plt.hist(nvt.Data_vel.ravel(), density=True, bins=100)

# FINISH THE CODE: START
# Kinetic partition function:
partition_function_KE =  #insert partition function here

# Probability as a function of velocity
def probability_KE(m, Temperature, v):
    return  #insert probability here

# FINISH THE CODE: END

v = np.arange(-5, 5, 0.001)
plt.plot(v, probability_KE(1, Temperature, v), 'r--', linewidth=3)
plt.xlabel(r'$\bf{v}\ \mathrm{(nm/ps)}$')
plt.ylabel(r'$\bf{P(v)}$')
plt.show()

# Self-Check: Should always equal 1!
print('The area under the curve is equal to: {:3.2f}'.format(np.trapz(probability_KE(1,Temperature,v), v)))

**Question 6: Calculate at which temperature half the particles would be able to cross the energy barrier and test your hypothesis using Molecular dynamics in the NVE ensemble. Comment on the result and compare with the mean speed of the Maxwell-Boltzmann distribution at the predicted temperature.**  

In [ ]:
##############################################
#                QUESTION 6                  #
#         Prediction of temperature          #
##############################################

L = # FINISH THE CODE: Insert lower integrating limit

T_list = np.arange(300,50000,100) # List of temperature
v_bot = np.arange(0, 10000, 0.1)  # Limits for calculating total Boltzmann weigth
v_top = np.arange(L, 10000, 0.1) # Limits for calculating the Boltzmann weigth of crossing
frac_list = []

# Integrand
def integrand(Temperature, v):
    return np.exp(-0.5*1*v**2/(nve.kB*Temperature))  # Kinetic boltzmann weigth

# Integrate using trapezoidal rule
for T in T_list:
    p_cross = np.trapz(integrand(T,v_top), v_top) # Integrate to get Boltzmann weigth of crossing
    p_tot = np.trapz(integrand(T,v_bot), v_bot)   # Integrate to get total boltzmann weigth
    frac = p_cross / p_tot                        # Fraction
    # Find temperature where fraction of the integrals is 0.5 i.e probability of half crossing
    if (abs(frac-0.5)) < 0.0008:
        print(r'The temperature at which half the particles can cross the barrier is {}'.format(T))
    frac_list.append(frac)
    
plt.plot(T_list, frac_list)

plt.xlabel("Temperature (Kelvin)")
plt.ylabel("Fraction of particles")
plt.show()

In [ ]:
# SIMULATION SETTINGS, MD-simulations (NVE)
Temperature =                                     # FINISH THE CODE: Insert calculated temperature
Nparticles = 10000                                # Number of independent particles
Tstep = 0.001                                     # [ps]
SimTime = 1                                       # Simulation time [ps]
Nstep = int(SimTime/Tstep)                        # Number of simulation steps

In [ ]:
# MD-simulations with NVT

nve = sim.Simulator(Nstep, Nparticles, Tstep, Temperature)

for j in range(nve.Nparticles):
    nve.Velocity[j] = nve.RandomVelocity(Temperature, 1)  # Set random velocity
    
for i in range(Nstep): # Main Loop for NVT simulation
    for j in range(nve.Nparticles): # Particle loop  
        nve.VelocityVerlet_NVE(j, Potential) # integrate motion of particle j
    nve.SampleData() # sample data for step i


In [ ]:
particle_id = np.arange(0, Nparticles, 1)
particle_id = np.stack((particle_id, [False]*Nparticles), axis=-1)
Tcounter = 0

# Check which particles has crossed the barrier
for particle in range(Nparticles):
    for position in nve.Data_pos.T[particle]:
        if position > 0.5:
            particle_id[particle][1] = True
            break

# Count the number of particles which has crossed the barrier
for particle in particle_id:
    if particle[1] == True:
        Tcounter += 1

print('{}% of the particles crossed the barrier at T = {} K'.format(Tcounter/Nparticles*100, Temperature))